In [12]:
#on va scrap google maps
#link : https://www.google.fr/maps/search/restaurant+paris/@48.8701745,2.2726075,18356m/data=!3m2!1e3!4b1?hl=fr&entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D
link="https://www.google.fr/maps/search/restaurant+paris/@48.8701745,2.2726075,18356m/data=!3m2!1e3!4b1?hl=fr&entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"

In [13]:
#xpath
#nom resto
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1

#note
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]

#prix
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span/span

#adresse
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]

#description
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[8]/button/div[1]


In [14]:
pip install selenium

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
import time
import random
import csv

In [16]:
# Configuration du WebDriver avec des options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--headless")

# User-Agent Rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
]
options.add_argument(f"user-agent={random.choice(user_agents)}")


In [17]:
def scrape_google_maps(link, n, output_file, arrondissement):
    """
    Fonction pour scraper les informations des restaurants à partir de Google Maps.

    :param link: URL de la recherche Google Maps
    :param n: Nombre de restaurants à scraper
    :param output_file: Nom du fichier CSV pour sauvegarder les résultats
    """
    # Initialisation du WebDriver
    driver = webdriver.Firefox(options=options)
    driver.get(link)

    # Attente pour charger la page et accepter les cookies
    time.sleep(5)
    try:
        cookies_button = driver.find_element(By.CSS_SELECTOR, ".VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)")
        cookies_button.click()
        time.sleep(2)
    except Exception as e:
        print("Pas de bouton cookies trouvé :", e)

    # Définir l'élément de scroll avec le XPath
    scroll_xpath = '//div[@class="XltNde tTVLSc"]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]'
    try:
        scrollable_div = driver.find_element(By.XPATH, scroll_xpath)
        print("Élément de scroll trouvé avec succès !")
    except Exception as e:
        print(f"Erreur lors de la recherche de l'élément de scroll : {e}")
        driver.quit()
        return

    # Fonction pour récupérer les informations d'un restaurant
    def get_restaurant_info():
        try:
            name = driver.find_element(By.CSS_SELECTOR, ".DUwDvf").text
        except:
            name = "NaN"
        try:
            rating = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(1) > span:nth-child(1)").text
        except:
            rating = "NaN"
        try:
            price = driver.find_element(By.CSS_SELECTOR, ".mgr77e > span:nth-child(1) > span:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text
        except:
            price = "NaN"
        try:
            restaurant_type = driver.find_element(By.CSS_SELECTOR, "button.DkEaL").text
        except:
            restaurant_type = "NaN"
        try:
            description = driver.find_element(By.CSS_SELECTOR, ".PYvSYb").text
        except:
            description = "NaN"
        try:
            address = driver.find_element(By.CSS_SELECTOR, "div.RcCsl:nth-child(3) > button:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text
        except:
            address = "NaN"

        # Ajouter l'URL Google Maps
        try:
        # Récupérer l'URL Google Maps directement depuis l'URL courante du navigateur
            google_maps_url = driver.current_url
        except:
            google_maps_url = "NaN"

        return {
            "name": name,
            "rating": rating,
            "price": price,
            "restaurant_type": restaurant_type,
            "description": description,
            "address": address,
            "google_maps_url": google_maps_url  # Ajouter l'URL Google Maps
        }


    # Collecte des données des restaurants
    restaurants_data = []
    restaurant_index = 1

    while restaurant_index <= n:
        try:
            # XPath dynamique pour sélectionner chaque restaurant
            restaurant_xpath = f"/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{(restaurant_index+1)*2-1}]/div/a"

            # Scrolling jusqu'à ce que l'élément soit trouvé
            max_scroll_attempts = 25 # Limite de scrolls pour éviter une boucle infinie
            scroll_attempts = 0
            while scroll_attempts < max_scroll_attempts:
                #si on est au 20eme scroll on attend 10sec
                if scroll_attempts == 24:
                    time.sleep(15)
                try:
                    restaurant = driver.find_element(By.XPATH, restaurant_xpath)
                    break  # Si l'élément est trouvé, on sort de la boucle
                except:
                    # Scroll vers le bas pour charger plus d'éléments
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
                    time.sleep(random.uniform(1, 2))  # Pause pour charger les nouveaux éléments
                    scroll_attempts += 1

            # Si l'élément n'est toujours pas trouvé après le scroll, arrêter
            if scroll_attempts >= max_scroll_attempts:
                print(f"Impossible de trouver le restaurant {restaurant_index} après plusieurs tentatives de scroll.")
                break

            # Cliquer sur le restaurant trouvé
            restaurant.click()
            time.sleep(3)  # Temps pour charger les informations

            # Récupération des informations
            info = get_restaurant_info()
            restaurants_data.append(info)
            print(f"Restaurant {restaurant_index} : {info}")

            # Retour à la liste
            driver.back()
            time.sleep(random.uniform(2, 4))  # Pause avant de continuer

            restaurant_index += 1
        except Exception as e:
            print(f"Erreur avec le restaurant {restaurant_index} : {e}")
            break

    # Filtrage des données par arrondissement
    def is_in_arrondissement(address, arrondissement):
        """
        Vérifie si l'adresse est dans l'arrondissement spécifié.

        :param address: L'adresse complète du restaurant
        :param arrondissement: Numéro de l'arrondissement (ex: "20", "16")
        :return: True si l'adresse est dans l'arrondissement, False sinon
        """
        if arrondissement == "16":
            # Vérifier pour les deux codes postaux du 16e arrondissement
            return "75016" in address or "75116" in address
        return f"750{arrondissement}" in address


    filtered_data = [
        restaurant for restaurant in restaurants_data
        if is_in_arrondissement(restaurant['address'], arrondissement)
    ]

    # Sauvegarde des données filtrées
    # Sauvegarde des données filtrées
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["name", "rating", "price", "restaurant_type", "description", "address", "google_maps_url"])
        writer.writeheader()
        writer.writerows(filtered_data)

    print(f"Données sauvegardées dans {output_file} avec filtrage pour l'arrondissement {arrondissement}")

    driver.quit()



In [18]:
n=100

In [19]:
#pour paris 1er arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+1er/@48.8631336,2.3262864,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_1er.csv"
scrape_google_maps(link, n, output_file, arrondissement='01')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Au Vieux Comptoir', 'rating': '4.6', 'price': '$$$', 'restaurant_type': 'French restaurant', 'description': 'Seasonally changing menus of French classics sprinkled with global touches in a convivial bistro.', 'address': '17 Rue des Lavandières Sainte-Opportune, 75001 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/plac

In [20]:
#pour paris 2eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+2eme/@48.867583,2.3312493,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_2eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='02')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Restaurant Clémentine - Bistrot Terrasse Paris 2', 'rating': '4.7', 'price': '$$', 'restaurant_type': 'French restaurant', 'description': 'Traditional French bistro dishes, plus regional wines, in a snug venue with pavement tables.', 'address': '5 Rue Saint-Marc, 75002 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/pl

In [21]:
#pour paris 3eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+3/@48.8675824,2.3312493,1993m/data=!3m1!1e3?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_3eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='03')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Terra', 'rating': '4.6', 'price': 'NaN', 'restaurant_type': 'Restaurant', 'description': 'High-end sharing plates & natural wines are served in this fashionable, evening-only restaurant.', 'address': '21 Rue des Gravilliers, 75003 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Terra/@48.8675824,2.3312493,2141m/d

In [22]:
#pour paris 4eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+4/@48.8632801,2.3384501,3987m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_4eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='04')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Au Bourguignon du Marais', 'rating': '4.6', 'price': '$$', 'restaurant_type': 'French restaurant', 'description': 'Escargot, beef bourguignon & other traditional Burgundy cuisine in a polished restaurant.', 'address': '52 Rue François Miron, 75004 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Au+Bourguignon+du+

In [23]:
#pour paris 5eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+5/@48.8633329,2.3384501,3987m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_5eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='05')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Bistrot La Grange - Restaurant Paris 5', 'rating': '4.6', 'price': '€10–20', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '17 Rue Lagrange, 75005 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Bistrot+La+Grange+-+Restaurant+Paris+5/@48.8508557,2.3125099,4283m/data=!3m1!1e3!4m10!1m2!2m1!1sres

In [24]:
#pour paris 6eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+6/@48.8676725,2.3003494,7973m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_6eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='06')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Paris Paris', 'rating': '4.4', 'price': '$$', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '8 Rue de Montfaucon, 75006 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Le+Paris+Paris/@48.8676625,2.3003494,8563m/data=!3m1!1e3!4m10!1m2!2m1!1srestaurant+paris+6!3m6!1s0x47e671bd225fbaef:0xa832a

In [25]:
#pour paris 7eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+7/@48.8484976,2.3185328,2211m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_7eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='07')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': "D'Chez Eux", 'rating': '4.6', 'price': 'NaN', 'restaurant_type': 'French restaurant', 'description': 'NaN', 'address': '2 Av. de Lowendal, 75007 Paris, France', 'google_maps_url': "https://www.google.fr/maps/place/D'Chez+Eux/@48.853413,2.2912112,2141m/data=!3m1!1e3!4m10!1m2!2m1!1srestaurant+paris+7!3m6!1s0x47e670263f2a386f:0xd1d5456c

In [ ]:
#pour paris 8eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+8/@48.8744256,2.2970739,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_8eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='08')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Sphère', 'rating': '4.8', 'price': '€100+', 'restaurant_type': 'Haute French restaurant', 'description': 'NaN', 'address': '18 Rue La Boétie, 75008 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Sph%C3%A8re/@48.874425,2.2970739,2141m/data=!3m2!1e3!5s0x47e66fc976f71e0f:0x9acef3ed0845fd65!4m10!1m2!2m1!1srestaurant

In [ ]:
#pour paris 9eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+9/@48.8744884,2.266174,7972m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_9eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='09')

In [ ]:
#pour paris 10eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+10/@48.8756274,2.3478646,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_10eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='10')

In [29]:
#pour paris 11eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+11/@48.8756901,2.3169646,7972m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_11eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='11')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'LE DIAMANT DE PARIS', 'rating': '4.5', 'price': '€30–40', 'restaurant_type': 'French restaurant', 'description': 'Intimate restaurant for classic French & Mediterranean dishes with occasional global twists.', 'address': '59 Rue Servan, 75011 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/LE+DIAMANT+DE+PARIS/@48.

In [30]:
#pour paris 12eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+12/@48.8756901,2.3169646,7972m/data=!3m1!1e3?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_12eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='12')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': "L'Aubergeade", 'rating': '4.8', 'price': '€30–40', 'restaurant_type': 'Restaurant', 'description': 'Typical, cozy bistro focusing on traditional French dishes like steak tartare, plus seafood.', 'address': '17 Rue Chaligny, 75012 Paris, France', 'google_maps_url': "https://www.google.fr/maps/place/L'Aubergeade/@48.8756901,2.3169646,8

In [ ]:
#pour paris 13eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+13/@48.8573678,2.2731491,7975m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_13eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='13')

In [32]:
#pour paris 14eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+14/@48.8329671,2.3124079,1995m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_14eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='14')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Chez Papa Montparnasse - Denfert 14ème', 'rating': '4.3', 'price': '€20–30', 'restaurant_type': 'French restaurant', 'description': 'NaN', 'address': '6 Rue Gassendi, 75014 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Chez+Papa+Montparnasse+-+Denfert+14%C3%A8me/@48.8329665,2.3124079,2142m/data=!3m1!1e3!4m10!1m

In [ ]:
#pour paris 15eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+15/@48.8330299,2.2815079,7979m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_15eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='15')

In [ ]:
#pour paris 16eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+16/@48.8331282,2.2815079,7979m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_16eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='16')

In [35]:
#pour paris 17eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+17/@48.8806927,2.2874628,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_17eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='17')

Pas de bouton cookies trouvé : Message: Unable to locate element: .VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Paris 17', 'rating': '4.7', 'price': '$$', 'restaurant_type': 'French restaurant', 'description': 'Traditional French plates presented in a cozy bistro with vintage charm & nostalgic prints.', 'address': '41 Rue Guersant, 75017 Paris, France', 'google_maps_url': 'https://www.google.fr/maps/place/Le+Paris+17/@48.881963,2.2705492,21

In [ ]:
#pour paris 18eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+18/@48.8807555,2.2565628,7971m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_18eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='18')

In [ ]:
#pour paris 19eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+19/@48.8890345,2.2902335,7970m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_19eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='19')

In [ ]:
#pour paris 20eme arrondssement:
link="https://www.google.fr/maps/search/restaurant+paris+20/@48.8628446,2.3749993,4420m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_20eme.csv"
scrape_google_maps(link, n, output_file, arrondissement='20')

In [39]:
import pandas as pd
#on ouvre les fichiers csv et on les concatene en un seul
#on enleve les doublons
#on ouvre tous les fichiers qui commencent par restaurants_paris
import glob
files = glob.glob("data_in_progress/restaurants_paris*.csv")
dataframes = []
for file in files:
    df = pd.read_csv(file)
    dataframes.append(df)
#concatenation
df = pd.concat(dataframes)
#enlever les doublons
df = df.drop_duplicates()
#sauvegarde
df.to_csv("restaurants_paris_with_url.csv", index=False)

In [ ]:
df.shape

In [41]:
#on affiche les differents types de restaurants
print(df["restaurant_type"].value_counts())

restaurant_type
French restaurant              138
Restaurant                     121
Bistro                          20
Italian restaurant              17
Brasserie                       11
Haute French restaurant          9
Fine dining restaurant           5
Mediterranean restaurant         5
Indian restaurant                4
Asian restaurant                 3
Chinese restaurant               3
Pizza restaurant                 3
Southwest France restaurant      3
Couscous restaurant              2
Thai restaurant                  2
Modern French restaurant         2
Seafood restaurant               2
Traditional restaurant           2
Vietnamese restaurant            2
Creperie                         2
Turkish restaurant               2
Basque restaurant                1
Hamburger restaurant             1
Hot pot restaurant               1
Modern European restaurant       1
Vegetarian restaurant            1
Lebanese restaurant              1
Meat dish restaurant             1
Lati